# **Bioinformatics Project - Computational Drug Discovery [Part 2] Exploratory Data Analysis**

In this Jupyter notebook, we will be building a real-life **data science project** that you can include in your **data science portfolio**. Particularly, we will be building a machine learning model using the ChEMBL bioactivity data.

In **Part 2**, we will be performing Descriptor Calculation and Exploratory Data Analysis.

---

## **Install conda and rdkit**

## **Load bioactivity data**

## **Calculate Lipinski descriptors**

### **Import libraries**

### **Calculate descriptors**

### **Combine DataFrames**

Let's take a look at the 2 DataFrames that will be combined.

### **Convert IC50 to pIC50**

### **Removing the 'intermediate' bioactivity class**
Here, we will be removing the ``intermediate`` class from our data set.

---

## **Exploratory Data Analysis (Chemical Space Analysis) via Lipinski descriptors**

### **Import library**

### **Frequency plot of the 2 bioactivity classes**

### **Scatter plot of MW versus LogP**

It can be seen that the 2 bioactivity classes are spanning similar chemical spaces as evident by the scatter plot of MW vs LogP.

### **Box plots**

## **Zip files**